In [17]:
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold 
from mordred import Calculator, descriptors
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import PandasTools
from sklearn.preprocessing import StandardScaler 
from sklearn.svm import SVR
datatra=pd.read_csv('BAF_train.csv',encoding = 'gbk')
dataext=pd.read_csv('BAF_test.csv',encoding = 'gbk')
y=datatra.iloc[:,1]
y2=dataext.iloc[:,1]
PandasTools.AddMoleculeColumnToFrame(datatra,'SMILES','Molecule')
PandasTools.AddMoleculeColumnToFrame(dataext,'SMILES','Molecule')
calc = Calculator(descriptors, ignore_3D=True)
X= pd.DataFrame(calc.pandas(datatra['Molecule']))
X2= pd.DataFrame(calc.pandas(dataext['Molecule']))
data1=pd.concat([X,X2])
data1=data1._get_numeric_data()
X=data1.iloc[:632,:]
X2=data1.iloc[632:]
scaler = StandardScaler()
X = scaler.fit_transform(X)
X2 = scaler.transform(X2) 
default_cv = KFold(n_splits=10, shuffle=True, random_state=666)

  4%|██▉                                                                              | 23/632 [00:02<00:46, 13.11it/s]

C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  6%|████▋                                                                            | 37/632 [00:03<00:31, 18.63it/s]

C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 10%|████████▏                                                                        | 64/632 [00:04<00:23, 24.36it/s]

C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 11%|█████████▎                                                                       | 18/157 [00:03<00:23,  5.86it/s]

C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\11936\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:07<00:00, 21.52it/s]


In [18]:
import numpy as np
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
RF=RandomForestRegressor(max_depth=10, max_features='sqrt',min_samples_leaf= 1,min_samples_split=2,n_estimators=1000,random_state=42)
SVR=SVR(C=1000, kernel='poly',max_iter= 10000,coef0=0,degree=2,gamma=0.0001)
XGB=xgb.XGBRegressor(n_estimators=700,
    max_depth=2,
    learning_rate=1,
    gamma=5,
    subsample=0.80,
    colsample_bytree=0.65)
GBR=GradientBoostingRegressor(learning_rate=0.0215,max_depth=10, max_features='sqrt',min_samples_leaf= 16,min_samples_split=10
                               ,n_estimators=750,random_state=42)


In [19]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
## 两两集成
stack1 = StackingCVRegressor(regressors=[RF,SVR],
                            meta_regressor = lr,
                            use_features_in_secondary=False)  # #
stack2 = StackingCVRegressor(regressors=[RF,GBR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack3 = StackingCVRegressor(regressors=[RF,XGB],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack4 = StackingCVRegressor(regressors=[XGB,SVR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack5 = StackingCVRegressor(regressors=[XGB,GBR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack6 = StackingCVRegressor(regressors=[GBR,SVR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #

In [20]:
## 三个集成
stack7 = StackingCVRegressor(regressors=[XGB,SVR,RF],
                            meta_regressor = lr,
                            use_features_in_secondary=False)  # #
stack8 = StackingCVRegressor(regressors=[XGB,SVR,GBR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack9 = StackingCVRegressor(regressors=[XGB,GBR,RF],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
stack10 = StackingCVRegressor(regressors=[GBR,SVR,RF],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #
#四个
stack11 = StackingCVRegressor(regressors=[XGB,SVR,RF,GBR],
                            meta_regressor = lr,
                            use_features_in_secondary=False )  # #

In [23]:
def fit_model(model):
    RANDOM_SEED = 42
    np.random.seed(RANDOM_SEED)
    model.fit(X,y) 
    fit = model.predict(X)
    pre = model.predict(X2)
    r2_tra = r2_score(y,fit)
    r2_ext = r2_score(y2, pre)
    rmse_tra = np.sqrt(mean_squared_error(y,fit))
    rmse_ext = np.sqrt(mean_squared_error(y2,pre))
    mae_tra = mean_absolute_error(y,fit)
    mae_ext = mean_absolute_error(y2,pre)
    return  r2_tra,r2_ext,rmse_tra, rmse_ext, mae_tra, mae_ext 

results = []
names = []
for model,label in zip([stack1,stack2,stack3,stack4,stack5,
                        stack6,stack7,stack8,stack9,stack10,stack11], 
                       ['1','2','3','4','5','6','7','8','9','10','11']): 
    names.append(label)
    score = fit_model(model)     #,xgb_re,'XGBoost',
    results.append(score)
results_df = pd.DataFrame(results,names)    


NameError: name 'cross_val_score' is not defined

In [26]:
from sklearn.model_selection import cross_val_score,KFold
def evaluate_model(model):
    cv = KFold(n_splits=10, shuffle=True, random_state=666)
    scores = cross_val_score(model, X, y, scoring='r2', cv=cv)
    #scores = np.sqrt(mean_squared_error(train_y.astype('int'),model.predict(train_x.values)))
    return scores
re = []
na = []
for model in [stack1,stack2,stack3,stack4,stack5,
                        stack6,stack7,stack8,stack9,stack10,stack11]: 
    score = evaluate_model(model)     #,xgb_re,'XGBoost',
    re.append(score)
    na.append(label)
    print("score: %0.3f (+/- %0.3f) "  % (score.mean(), score.std()))
plt.boxplot(re, labels=na, showmeans=True,showfliers= False,vert=True)


#放在一起比差异太小啦！
re_all = results + re
na_all = names + na
plt.boxplot(re_all, labels=na_all, showmeans=True,showfliers= False,vert=True)


score: 0.819 (+/- 0.075) 
score: 0.836 (+/- 0.076) 
score: 0.806 (+/- 0.073) 
score: 0.785 (+/- 0.097) 
score: 0.845 (+/- 0.068) 
score: 0.851 (+/- 0.067) 
score: 0.826 (+/- 0.074) 
score: 0.851 (+/- 0.066) 
score: 0.840 (+/- 0.069) 
score: 0.851 (+/- 0.067) 
score: 0.854 (+/- 0.068) 


NameError: name 'plt' is not defined

In [27]:
results_df.to_csv("jicheng-BAF.csv")

In [28]:
pred1=pd.DataFrame(stack9.predict(X))
pred2=pd.DataFrame(stack9.predict(X2))
pred1.to_csv("stack9-BAF-train.csv")
pred2.to_csv("stack9-BAF-test.csv")